In [1]:
import numpy as np
import tensorflow as tf
from config import Config
import utils3
from layer import *
import sys
import os
import pandas as pd
import pickle
import math
import time
import single_atn_rnn as sa

In [2]:
config = Config(words_num=60, des_words_num=60, batch_size=256, eval_batch_size=300, hidden_size=256, lr=0.01, 
                reg_rate=0.00001, epoch_num=20, save_per_step=500, eval_per_step=50, keep_prob=0.5, 
                atn_hidden_size=256, shuffle=True, summary_dir="./Log/", ckpt_path="./models/",
                dataset_dir="./ieee_zhihu_cup/", num_sentences=2)
vocab = pickle.load(open('./ieee_zhihu_cup/vocab.dict' ,'r'))
utils = utils3.Utils(config, vocab)

In [3]:
class Zhihu_Classifier(object):
    def __init__(self, config, sess):
        self.config = config
        self.batch_size = config.batch_size
        self.eval_batch_size = config.eval_batch_size
        self.hidden_size = config.hidden_size
        self.lr = config.lr
        self.epoch_num = config.epoch_num
        self.reg_rate = config.reg_rate
        self.words_num = config.words_num
        self.des_words_num = config.des_words_num
        self.save_per_step = config.save_per_step
        self.eval_per_step = config.eval_per_step
        self.ckpt_path = config.ckpt_path
        self.keep_prob = config.keep_prob
        self.summary_dir = config.summary_dir
        self.atn_hidden_size = config.atn_hidden_size
        self.shuffle = config.shuffle
        self.max_score = 0.0
        self.dataset_dir = config.dataset_dir
        self.num_sentences = config.num_sentences
        
        self.embed_size = 
        self.sess = sess
        self.istrain = True
    
    def train(self):
        # 生成数据
        batches = utils.generate_t_batch()
        eval_batches = utils.generate_e_batch()
        
        model = sa.HierarchicalAttention(1999, self.lr, self.batch_size, 1000, 0.9, self.words_num*2, 
                                         self.num_sentences, 570783, self.embed_size, self.hidden_size, 
                                         self.istrain, multi_label_flag=True)
        self.sess.run(tf.global_variables_initializer())
        self.predict_top_5 = tf.nn.top_k(model.logits, k=5)
        counter = 0
        for batch in batches:
            counter = counter + 1
#             print(counter)
            x_batch, x_des, a, b, y_batch = zip(*batch)
            x_batch = np.concatenate((x_batch, x_des), axis=1)
            self.batch_length = len(x_batch)
            self.input_x = x_batch
            
            feed_dict = {model.input_x: self.input_x, 
                         model.dropout_keep_prob: self.keep_prob,
                         model.batch_length:[self.batch_length*self.num_sentences*self.hidden_size],
                         model.sen_batch_length:[self.batch_length*self.hidden_size*2],
                         model.input_y_multilabel:y_batch}
            train_loss, _= self.sess.run([model.loss_val, model.train_op], feed_dict) 
            
            if (counter % self.eval_per_step == 0):
                print ("Step:",counter)
                print ("Train loss:",train_loss)
                x_eval_batch, x_des, a, b, y_eval_batch, real_labels = zip(*(eval_batches.next()))
                x_eval_batch = np.concatenate((x_eval_batch, x_des), axis=1)
                self.eval_input_x = x_eval_batch
                self.do_eval(model, self.eval_input_x, y_eval_batch, real_labels)
                
            
    # 在验证集上做验证，报告损失、精确度
    def do_eval(self, model, evalX, evalY, real_labels):
        self.eval_batch_length = len(evalX)
        feed_dict = {model.input_x: evalX,
                     model.input_y_multilabel:evalY,
                     model.dropout_keep_prob: 1.0, 
                     model.batch_length:[self.eval_batch_length*self.num_sentences*self.hidden_size],
                     model.sen_batch_length:[self.eval_batch_length*self.hidden_size*2],
                    }
        curr_eval_loss, predict_5 = self.sess.run([model.loss_val, self.predict_top_5], feed_dict)
        print ("Evaluation loss:",curr_eval_loss)
#                     print ("real_labels:",real_labels[:5])
#                     print ("predict:",predict_5[1][:5])
#                     print ("predict:",predict_5[0][:5])
        predict_label_and_marked_label_list = []
        for predict,label in zip(predict_5[1],real_labels):
            predict_label_and_marked_label_list.append((list(predict),list(label)))
        score = utils.eval(predict_label_and_marked_label_list)
        print("score:",score)    

In [4]:
with tf.Session() as sess:
    classifier = Zhihu_Classifier(config, sess)
    classifier.train()

going to use multi label loss.
('sigmoid_cross_entropy_with_logits.losses:', <tf.Tensor 'loss/logistic_loss:0' shape=(?, 1999) dtype=float32>)


ResourceExhaustedError: OOM when allocating tensor with shape[570783,256]
	 [[Node: OptimizeLoss/train/update_Embedding/mul_2 = Mul[T=DT_FLOAT, _class=["loc:@Embedding"], _device="/job:localhost/replica:0/task:0/gpu:0"](OptimizeLoss/Embedding/Adam/read, OptimizeLoss/train/beta1)]]
	 [[Node: OptimizeLoss/train/update/_40 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_42181_OptimizeLoss/train/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'OptimizeLoss/train/update_Embedding/mul_2', defined at:
  File "/home/jasperyang/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/jasperyang/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/jasperyang/anaconda2/lib/python2.7/site-packages/ipyke

In [23]:
batches = utils.generate_e_batch()

In [25]:
a,b,c = zip(*(batches.next()))